In [1]:
import pandas as pd
import re
from langdetect import detect

# Đọc file dữ liệu
books = pd.read_csv('dataset/books.csv')
ratings = pd.read_csv('dataset/ratings.csv')

print(ratings.head())

                 User_id  book_id  review/score
0  A00109803PZJ91RLT7DPN   134046           5.0
1  A00117421L76WVWG4UX95    84912           5.0
2   A0015610VMNR0JC9XVL1    51730           5.0
3   A0015610VMNR0JC9XVL1    65628           5.0
4   A0015610VMNR0JC9XVL1   105456           5.0


In [ ]:
import pandas as pd
import re
from langdetect import detect

# Đọc file dữ liệu
books = pd.read_csv('dataset/books.csv')
ratings = pd.read_csv('dataset/ratings.csv')

print(ratings.head())

                 User_id  book_id  review/score
0  A00109803PZJ91RLT7DPN   134046           5.0
1  A00117421L76WVWG4UX95    84912           5.0
2   A0015610VMNR0JC9XVL1    51730           5.0
3   A0015610VMNR0JC9XVL1    65628           5.0
4   A0015610VMNR0JC9XVL1   105456           5.0


In [2]:
# Tính số lượng của mỗi User_id
user_rating_counts = ratings['User_id'].value_counts()

# Đếm số lượng user có số đánh giá > 20
users_with_more_than_20_ratings = len(user_rating_counts[user_rating_counts > 50])

print("Số lượng user có số đánh giá > 20:", users_with_more_than_20_ratings)


Số lượng user có số đánh giá > 20: 2074


In [3]:
# Tính số lượng của mỗi User_id
user_rating_counts = ratings['User_id'].value_counts()

# Lọc ra những User_id có số đánh giá > 20
users_with_more_than_20_ratings = user_rating_counts[user_rating_counts > 50].index

# Lọc ratings chỉ giữ lại những hàng có User_id nằm trong danh sách trên
filtered_ratings = ratings[ratings['User_id'].isin(users_with_more_than_20_ratings)]
ratings = filtered_ratings
print(ratings)

               User_id  book_id  review/score
2811     A106016KSI0YQ      528           5.0
2812     A106016KSI0YQ     2457           4.0
2813     A106016KSI0YQ     3407           2.0
2814     A106016KSI0YQ     4191           3.0
2815     A106016KSI0YQ     6609           4.0
...                ...      ...           ...
2113015  AZSV99SDJC242   201547           4.0
2113016  AZSV99SDJC242   201753           5.0
2113017  AZSV99SDJC242   205200           4.0
2113018  AZSV99SDJC242   205262           4.0
2113019  AZSV99SDJC242   207728           4.0

[253226 rows x 3 columns]


In [4]:
import pandas as pd
import re

books = books.sample(n=3000, random_state=2)

ratings = ratings.merge(books[['book_id']], on='book_id', how='inner')

print(books)
print(ratings)

        book_id                                              Title  \
78458    117174  The two thousand isles;: A short account of th...   
41039     56697                        Noise control for engineers   
117463   171425  The Godless and the Infidels: Memoirs of a Ras...   
79366    118463  Fetish: Masterpieces of Erotic Fantasy Photogr...   
5006       7061     Raw Faith: Nurturing the Believer in All of Us   
...         ...                                                ...   
61220     92712                                  In a Paris Moment   
44426     61454                               Chance of a Lifetime   
73244    109705                                     A Perfect Life   
43275     59758                                        O Jerusalem   
22959     31608               Life, Sex, and Everything in Between   

                                              description  \
78458   a short and classic account of the authors tra...   
41039   this classic and authoritativ

In [5]:
from sklearn.model_selection import train_test_split

# Chia dữ liệu ratings thành hai tập train và test
train, test = train_test_split(ratings, test_size=0.2, random_state=42)

# Kiểm tra kích thước của mỗi tập
print("Kích thước tập train:", train.shape)
print("Kích thước tập test:", test.shape)


Kích thước tập train: (3380, 3)
Kích thước tập test: (846, 3)


In [6]:
ratings = train

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import jaccard_score
import pandas as pd
import numpy as np

tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(books['description'])
cosine_sim_description = cosine_similarity(tfidf_matrix)
cosine_sim_description_df = pd.DataFrame(cosine_sim_description, index=books['book_id'], columns=books['book_id'])
print(cosine_sim_description_df)


book_id    117174    56697     171425    118463    7061      44087     185025  \
book_id                                                                         
117174   1.000000  0.009563  0.020222  0.000000  0.000000  0.000000  0.000000   
56697    0.009563  1.000000  0.004336  0.000000  0.000000  0.000000  0.000000   
171425   0.020222  0.004336  1.000000  0.000000  0.000000  0.000000  0.000000   
118463   0.000000  0.000000  0.000000  1.000000  0.000000  0.000000  0.000000   
7061     0.000000  0.000000  0.000000  0.000000  1.000000  0.000000  0.000000   
...           ...       ...       ...       ...       ...       ...       ...   
92712    0.021292  0.000000  0.024874  0.000000  0.000000  0.003806  0.000000   
61454    0.000000  0.006412  0.006144  0.000000  0.000000  0.002924  0.000000   
109705   0.007002  0.023375  0.022326  0.006035  0.071458  0.003528  0.003612   
59758    0.040650  0.000000  0.018324  0.000000  0.000000  0.000000  0.000000   
31608    0.000000  0.030002 

In [8]:
mlb_categories = MultiLabelBinarizer()
mlb_authors = MultiLabelBinarizer()
categories_encoded = mlb_categories.fit_transform(books['categories'])
authors_encoded = mlb_authors.fit_transform(books['authors'])


In [9]:
# movies['Genres'] = movies['Genres'].apply(lambda x: x.split('|'))


# mlb_genres = MultiLabelBinarizer()
# genres_encoded = mlb_genres.fit_transform(movies['Genres'])


In [10]:
# def check_all_zero(encoded_matrix, feature_name, df):
#     all_zero_rows = []
#     for idx, row in enumerate(encoded_matrix):
#         if np.all(row == 0):
#             all_zero_rows.append(idx)
    
#     if all_zero_rows:
#         print(f"Các hàng toàn 0 trong {feature_name}: {all_zero_rows}")
#         zero_id_list = df.iloc[all_zero_rows]['book_id'].tolist()  # Giả sử cột ID có tên là 'id'
#         print(f"Các ID của sách có hàng mã hóa toàn 0 trong {feature_name}: {zero_id_list}")
#     else:
#         print(f"Không có hàng nào toàn 0 trong {feature_name}.")

# check_all_zero(categories_encoded, "categories", books)
# check_all_zero(authors_encoded, "authors", books)

Khoảng cách L2 (Euclidean Distance)

In [11]:
# import pandas as pd
# import numpy as np
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity

# tfidf_categories = TfidfVectorizer()
# tfidf_authors = TfidfVectorizer()

# # Chuyển đổi dữ liệu của cột 'categories' và 'authors' thành TF-IDF vectors
# categories_encoded = tfidf_categories.fit_transform(books['categories'])
# authors_encoded = tfidf_authors.fit_transform(books['authors'])

similarity_categories = cosine_similarity(categories_encoded)
similarity_authors = cosine_similarity(authors_encoded)

similarity_categories_df = pd.DataFrame(similarity_categories, index=books['book_id'], columns=books['book_id'])
similarity_authors_df = pd.DataFrame(similarity_authors, index=books['book_id'], columns=books['book_id'])



In [12]:
# import pandas as pd
# import numpy as np
# import faiss

# # Hàm tính toán ma trận tương đồng bằng FAISS
# def faiss_similarity(encoded_matrix):
#     encoded_matrix = encoded_matrix.astype(np.float32)
#     index = faiss.IndexFlatL2(encoded_matrix.shape[1])
#     index.add(encoded_matrix)
#     distances, _ = index.search(encoded_matrix, k=encoded_matrix.shape[0])
#     return 1 - (distances / np.max(distances))

# # Tính toán ma trận tương đồng cho thể loại
# similarity_categories = faiss_similarity(categories_encoded)

# # Tính toán ma trận tương đồng cho tác giả
# similarity_authors = faiss_similarity(authors_encoded)

# # Chuyển thành DataFrame
# similarity_categories_df = pd.DataFrame(similarity_categories, index=books['book_id'], columns=books['book_id'])
# similarity_authors_df = pd.DataFrame(similarity_authors, index=books['book_id'], columns=books['book_id'])
# # Xuất kết quả
# print("Ma trận tương đồng cho thể loại:")
# print(similarity_categories_df)
# print("Ma trận tương đồng cho tác giả:")
# print(similarity_authors_df)



In [13]:
alpha = 0.2  # (description)
beta = 0.4  # (categories)
gamma = 0.4  # (authors)

similarity_matrix = (
    alpha * cosine_sim_description_df +
    beta * similarity_categories_df +
    gamma * similarity_authors_df
) 

print(similarity_matrix)

book_id    117174    56697     171425    118463    7061      44087     185025  \
book_id                                                                         
117174   1.000000  0.423498  0.481071  0.531143  0.415255  0.477027  0.549366   
56697    0.423498  1.000000  0.473417  0.422218  0.512632  0.447931  0.449683   
171425   0.481071  0.473417  1.000000  0.430964  0.509187  0.633333  0.677174   
118463   0.531143  0.422218  0.430964  1.000000  0.411731  0.460778  0.462899   
7061     0.415255  0.512632  0.509187  0.411731  1.000000  0.541213  0.446655   
...           ...       ...       ...       ...       ...       ...       ...   
92712    0.312941  0.472550  0.338308  0.401150  0.434998  0.367428  0.309717   
61454    0.548958  0.440521  0.593383  0.421607  0.529905  0.688815  0.617882   
109705   0.450422  0.477224  0.671132  0.461985  0.491453  0.634039  0.627501   
59758    0.641003  0.493895  0.483426  0.480978  0.417882  0.479761  0.500601   
31608    0.424658  0.428818 

In [14]:
all_book_ids = books['book_id'].unique().tolist()
print(all_book_ids)

[117174, 56697, 171425, 118463, 7061, 44087, 185025, 103679, 126007, 114632, 133869, 40800, 93885, 83824, 148332, 41296, 135971, 63961, 144766, 183383, 84918, 115305, 117066, 189269, 178710, 30305, 189492, 37798, 110504, 136808, 99896, 98950, 147147, 182037, 126624, 156035, 140872, 124128, 48218, 29479, 27500, 109299, 181895, 8599, 10021, 92600, 89384, 111295, 34769, 169286, 203445, 136076, 90162, 34855, 183694, 142535, 20841, 23706, 141077, 19445, 22249, 69830, 155806, 186570, 125299, 60081, 38127, 181999, 201543, 79483, 132033, 1077, 180608, 165167, 177042, 129507, 150943, 9892, 39454, 210297, 178005, 131281, 164790, 69061, 69299, 151628, 113878, 81389, 75374, 201213, 190922, 128845, 152743, 34590, 149037, 124978, 196371, 128031, 31395, 96072, 141048, 122146, 18963, 15768, 172537, 178172, 207093, 168800, 18364, 68477, 116981, 131457, 23125, 100888, 201258, 90968, 150277, 33280, 143468, 80763, 209282, 34606, 20673, 4219, 148012, 187515, 138631, 38627, 3975, 123246, 170694, 112751, 590

In [15]:
import pandas as pd

def check_index_matches(df, book_ids):
    # Chuyển chỉ mục của DataFrame thành một tập hợp
    df_index_set = set(df.index)
    # Chuyển book_ids thành một tập hợp
    book_ids_set = set(book_ids)
    
    # Kiểm tra xem hai tập hợp có bằng nhau không
    if df_index_set == book_ids_set:
        return True
    else:
        return False

result = check_index_matches(similarity_matrix, all_book_ids)
print("Chỉ mục của df trùng khớp với book_ids:", result) 


Chỉ mục của df trùng khớp với book_ids: True


In [16]:
user_rating_matrix = ratings.pivot_table(index="User_id", columns="book_id", values="review/score")

In [17]:
def calculate_similarity(user_id, book_id, rated_books):
    if book_id not in similarity_matrix.index:
        return None

    similarities = similarity_matrix.loc[book_id]  

    ratings_for_rated_books = user_rating_matrix.loc[user_id]

    if ratings_for_rated_books.empty:
        return None

    weighted_sum = 0
    similarity_sum = 0
    for rated_books in rated_books:
        weighted_sum = weighted_sum + (similarities.loc[rated_books]*ratings_for_rated_books[rated_books])
        similarity_sum += np.abs(similarities.loc[rated_books])
   

    if similarity_sum > 0:
        average_similarity = weighted_sum / similarity_sum
        return average_similarity
    
    return None

In [18]:
# def calculate_similarity(user_id, movie_id, rated_movies):
#     if movie_id not in similarity_matrix.index:
#         return None

#     similarities = similarity_matrix.loc[movie_id]  

#     ratings_for_rated_movies = user_rating_matrix.loc[user_id]

#     if ratings_for_rated_movies.empty:
#         return None

#     count = 0
#     similarity_sum = 0
#     for rated_movie in rated_movies:
#         if ratings_for_rated_movies[rated_movie] >3:
#             similarity_sum += np.abs(similarities.loc[rated_movie])
#             count+=1
    
#     if count == 0:
#         return None
    
#     similarity = similarity_sum/count
#     return similarity * 5

In [19]:
def recommend_books(user_id, top_k=10):
    rated_books = ratings[ratings['User_id'] == user_id]['book_id'].unique()
    # rated_books = [book_id for book_id in rated_books if book_id in similarity_matrix.index]

    if len(rated_books) == 0:
        print(f"Người dùng {user_id} chưa đánh giá sách nào.")
        return []

    similarity_scores = {}

    # test_books = test_data['book_id'].unique()
    # unrated_books = books_data_df['book_id'][~books_data_df['book_id'].isin(rated_books)]
    # unrated_books = unrated_books[unrated_books.isin(test_books)]


    # for book_id in unrated_books:
    #     similarity_score = calculate_similarity(book_id, rated_books)
        
    #     if similarity_score is not None:
    #         similarity_scores[book_id] = similarity_score

    for book_id in all_book_ids:
        if(book_id not in rated_books):
            similarity_score = calculate_similarity(user_id, book_id, rated_books)

            if similarity_score is not None:
                similarity_scores[book_id] = similarity_score

    if similarity_scores:
        suggested_books = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)[:top_k]

        return [[book[0], similarity_scores[book[0]]] for book in suggested_books]
    else:
        print(f"Không có sách nào để gợi ý cho người dùng {user_id}.")
        return []



In [20]:
user_id = "A2VE83MZF98ITY"
top_k = 10

suggested_books = recommend_books(user_id, top_k)
for book_id, similarity_score in suggested_books:
    print(book_id, end=" ")  # Print each book_id followed by a 
    
print()
if suggested_books:
    for book_id, similarity_score in suggested_books:
        # Fetch book details from the books DataFrame
        book_info = books.loc[books['book_id'] == book_id, ['Title', 'categories', 'description', 'authors']]
        
        if not book_info.empty:
            title = book_info['Title'].values[0]
            categories = book_info['categories'].values[0]
            description = book_info['description'].values[0]
            authors = book_info['authors'].values[0]
            
            # Print book information
            print(f"Sách: '{title}' (ID: {book_id})")
            print(f"Thể loại: {categories}")
            print(f"Tác giả: {authors}")
            print(f"Mô tả: {description}")
            print(f"Điểm tương đồng: {similarity_score:.4f}")
            print("-" * 50)
        else:
            print(f"Không tìm thấy sách với ID: {book_id}")
else:
    print("Không có cuốn sách nào để gợi ý.")


158712 204243 135996 165897 163532 173854 113522 75850 55911 141173 
Sách: 'Practical Approach to the Study of Form in Music' (ID: 158712)
Thể loại: ['Music']
Tác giả: ['Peter Spencer', 'Peter M. Temko']
Mô tả: learning musics organizing principles    approaching the study of form as an exercise in perceiving the interaction of a number of discrete musical events spencer and temkos book embodies much more than a search for visual clues students of form develop perceptual tools that allow them to proceed from the aural experience to an understanding of the archprinciples upon which music is organized the authors hold that the organizing principles of a given piece of music may be gleaned from studying the internal attributes that give a section its specific identity the functional relations between sections and the ordering of those sections
Điểm tương đồng: 4.5072
--------------------------------------------------
Sách: 'Robust Control Systems: Theory and Case Studies' (ID: 204243)
Thể

In [21]:
def get_books_for_user(user_id):
    # Filter ratings to get books rated by the specified user_id
    user_ratings = test[test['User_id'] == user_id]
    
    # Merge with books DataFrame to get book details including title, categories, description, and authors
    user_books = user_ratings.merge(books[['book_id']], on='book_id', how='inner')
    
    # Return the list of book_ids
    book_ids = user_books['book_id'].tolist()
    
    return book_ids

# Example usage
user_id = "A1K1JW1C5CUSUZ"
list_books = get_books_for_user(user_id)
print(list_books)
if list_books:  # Ensure that the list of book_ids is not empty
    for book_id in list_books:
        # Fetch book details from the books DataFrame
        book_info = books.loc[books['book_id'] == book_id, ['Title', 'categories', 'description', 'authors']]
        
        if not book_info.empty:
            title = book_info['Title'].values[0]
            categories = book_info['categories'].values[0]
            description = book_info['description'].values[0]
            authors = book_info['authors'].values[0]
            
            # Print book information
            print(f"Sách: '{title}' (ID: {book_id})")
            print(f"Thể loại: {categories}")
            print(f"Tác giả: {authors}")
            print(f"Mô tả: {description}")
            print("-" * 50)
        else:
            print(f"Không tìm thấy sách với ID: {book_id}")
else:
    print("Không có cuốn sách nào để gợi ý.")


[55134, 69587, 17522, 146960, 163590]
Sách: 'The Math Gene: How Mathematical Thinking Evolved and Why Numbers Are Like Gossip' (ID: 55134)
Thể loại: ['Science']
Tác giả: ['Keith Devlin']
Mô tả: why is math so hard and why despite this difficulty are some people so good at it if theres some inborn capacity for mathematical thinkingwhich there must be otherwise no one could do it why cant we all do it well keith devlin has answers to all these difficult questions and in giving them shows us how mathematical ability evolved why its a part of language ability and how we can make better use of this innate talenthe also offers a breathtakingly new theory of language developmentthat language evolved in two stages and its main purpose was not communicationto show that the ability to think mathematically arose out of the same symbolmanipulating ability that was so crucial to the emergence of true language why then cant we do math as well as we can speak the answer says devlin is that we can and

In [22]:
user_ids = test['User_id'].unique().tolist()

print("Danh sách UserID độc nhất:")
print(user_ids)


Danh sách UserID độc nhất:
['A14OJS0VWMOSWO', 'A1X8VZWTOG8IS6', 'A188JKV8QVVOT7', 'AD8UQ909Z49UK', 'A3M3W5GV0HIQK4', 'A28CCN6VYHRUKS', 'A3IKBHODOTYYHM', 'APOGXH3A74CM3', 'A280GY5UVUS2QH', 'A22DUZU3XVA8HA', 'ARBMZ59SXGUGI', 'A592JDESZFMNF', 'A1S52XH2CS4Y26', 'A20EEWWSFMZ1PN', 'A9LAPV8XNKZVZ', 'A2K40JBQHJY1HH', 'A24IA3AVBZMDCX', 'A25KKXUQQ0OLWB', 'A1NATT3PN24QWY', 'AXQ8T2D9IT6UG', 'A1A3ATZCVOKP11', 'A1EKTLUL24HDG8', 'A3OJFPKMCXKOM0', 'A1KXONFPU2XQ5K', 'A34UTL4AVX80MK', 'A2VKWLCNZF4ZVB', 'A1MC6BFHWY6WC3', 'A3C6CZC2JP67VK', 'ATNQJL1SF21TS', 'A1FSJH67CYP9Y2', 'A2EP0QO57Y9YW6', 'A3B48AN8JID1JK', 'A1IPM23IJ8Q3L4', 'A1XJ64FBB2IRGT', 'A2OJW07GQRNJUT', 'A1SCJWCMQ3W3KK', 'A1NGKKAMKCZHHL', 'A3GGUBZMTQD1B9', 'A3AUL23GMCOP2A', 'A55MRYPUAX4QU', 'ARJIAB8SLWAOU', 'A3UPYGJKZ0XTU4', 'A27CFNHYZG6WS8', 'A3QVAKVRAH657N', 'A1DQZBHEU98PBT', 'A2CW9IQAPFEYLM', 'A302HGMTLMEM2I', 'AT2R7ZVT7QYOU', 'A1KE3JBO3KQX2O', 'A2QT0KPQU671OU', 'A3OGMDFM045MQ7', 'A2QZQBINBG6B5N', 'AFVQZQ8PW0L', 'A3DGB4HN3QCGAL', 'A20MN959ZZH2

In [23]:
import random

user_ids_sample = random.sample(user_ids, 500)
user_ids = user_ids_sample
print("Danh sách 1000 UserID ngẫu nhiên:")
print(user_ids)


Danh sách 1000 UserID ngẫu nhiên:
['A2YXRT2XIJIO57', 'A2YTHMJJJQCOZ', 'A3TEH90X39WC8F', 'AG7R3MMF8QLDT', 'A3EX36SNRYD5VL', 'A2F3M93RRLFQNJ', 'A19B0WUZCH8I4R', 'A1RMHZSWZ7ZEQO', 'A3D6KZT0QG6UKB', 'A2PNTEDGMOLMOY', 'A15YPJLKLZS0MK', 'A2CR57GAJKNWVV', 'A9TG2NFA614S9', 'A1U7L28N1N8E0L', 'A2BIAR1V0PWO27', 'A2K9FXOFFHVNWG', 'A2NTJUI2DLZF4R', 'AUTBHG6070SL4', 'A1XTWXIMUCDGQE', 'A3SN9EF7GTNEU2', 'A3SQQYS79AV13M', 'A3VIOCJZ22JZXT', 'AAP7PPBU72QFM', 'A3R90OGE1EACZE', 'A1JH0MLDCBI9HW', 'A1XSKZN07IG780', 'A1VJU3NG5W15L4', 'A3B48AN8JID1JK', 'A3QGRSZ7VTV0SY', 'A1IU7S4HCK1XK0', 'A1AXHNRDJEQ0B', 'A11SBLG65E39RR', 'A2GNGJVLZ881JE', 'A27CFNHYZG6WS8', 'A1CHOKV10NEI8X', 'A17BT73RVRQYSP', 'AHEW91ZUQDSKR', 'A2VZ11U5DXM8J5', 'A3VBVDOMJT49GH', 'A2BSZPRBKYAB2K', 'A3QBCX2QMKJ7AH', 'A2IKY4WH4E11AI', 'AZ216MK9KO1S0', 'A3GA09FYFKL4EY', 'AKI1QGJUENQWX', 'ADJBUPS8SJHTC', 'A3FS107ZKRPQYR', 'A3KUKRMJ79VQI4', 'A4R8GJX3UZ1I', 'A3N6JG64P2G9EV', 'A172JDGLV9FFX', 'A1NT7ED5TATUAM', 'A2VKWLCNZF4ZVB', 'A3M77GPST0EICR', 'A1R6D

In [24]:
import numpy as np

total_tp = 0  # Tổng số True Positives
total_fp = 0  # Tổng số False Positives
total_fn = 0  # Tổng số False Negatives
total_squared_error = 0  # Tổng bình phương sai số
total_absolute_error = 0  # Tổng sai số tuyệt đối
total_predictions = 0  # Tổng số dự đoán được sử dụng cho MSE, RMSE, MAE

count = 0

for user_id in user_ids:
    count +=1
    print(f"Bắt đầu tính user thứ {count}")
    actual_books = test[test['User_id'] == user_id][['book_id', 'review/score']].values.tolist()
    
    suggested_books = recommend_books(user_id, top_k=10) 
    
    actual_dict = dict(actual_books)
    
    suggested_book_ids = set([book_id for book_id, _ in suggested_books])
    actual_book_ids = set([book_id for book_id, _ in actual_books])

    # TP (True Positives), FP (False Positives) và FN (False Negatives) cho use hiện tại
    tp = len(suggested_book_ids.intersection(actual_book_ids))
    fp = len(suggested_book_ids - actual_book_ids)
    fn = len(actual_book_ids - suggested_book_ids)
    
    total_tp += tp
    total_fp += fp
    total_fn += fn

    # MSE, RMSE, MAE
    for book_id, predicted_score in suggested_books:
        if book_id in actual_dict:
            actual_score = actual_dict[book_id]
            total_squared_error += (actual_score - predicted_score) ** 2
            total_absolute_error += abs(actual_score - predicted_score)
            total_predictions += 1

precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

mse = total_squared_error / total_predictions if total_predictions > 0 else 0
rmse = np.sqrt(mse)
mae = total_absolute_error / total_predictions if total_predictions > 0 else 0

print("Overall Precision:", precision)
print("Overall Recall:", recall)
print("Overall F1-score:", f1_score)
print("Overall MSE:", mse) 
print("Overall RMSE:", rmse)
print("Overall MAE:", mae)


Bắt đầu tính user thứ 1
Bắt đầu tính user thứ 2
Bắt đầu tính user thứ 3
Bắt đầu tính user thứ 4
Bắt đầu tính user thứ 5
Người dùng A3EX36SNRYD5VL chưa đánh giá sách nào.
Bắt đầu tính user thứ 6
Bắt đầu tính user thứ 7
Người dùng A19B0WUZCH8I4R chưa đánh giá sách nào.
Bắt đầu tính user thứ 8
Bắt đầu tính user thứ 9
Bắt đầu tính user thứ 10
Bắt đầu tính user thứ 11
Người dùng A15YPJLKLZS0MK chưa đánh giá sách nào.
Bắt đầu tính user thứ 12
Bắt đầu tính user thứ 13
Người dùng A9TG2NFA614S9 chưa đánh giá sách nào.
Bắt đầu tính user thứ 14
Người dùng A1U7L28N1N8E0L chưa đánh giá sách nào.
Bắt đầu tính user thứ 15
Người dùng A2BIAR1V0PWO27 chưa đánh giá sách nào.
Bắt đầu tính user thứ 16
Bắt đầu tính user thứ 17
Bắt đầu tính user thứ 18
Bắt đầu tính user thứ 19
Bắt đầu tính user thứ 20
Bắt đầu tính user thứ 21
Bắt đầu tính user thứ 22
Người dùng A3VIOCJZ22JZXT chưa đánh giá sách nào.
Bắt đầu tính user thứ 23
Bắt đầu tính user thứ 24
Bắt đầu tính user thứ 25
Bắt đầu tính user thứ 26
Bắt đầu tí